In [13]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SQLContext
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.sql.Row
import org.apache.spark.ml.feature.{HashingTF, Tokenizer,StopWordsRemover,IDF}
import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import com.ibm.analytics.ngp.ingest.Connectors
import org.apache.spark.sql.hive.HiveContext
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}

In [14]:
import org.apache.spark.sql.SparkSession

// @hidden_cell
// This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
// You might want to remove those credentials before you share your notebook.
def setHadoopConfig9433696f0f574c02aa81541a1538139f(name: String) = {
    // This function sets the Hadoop configuration so it is possible to
    // access data from Bluemix Object Storage using Spark

    val prefix = "fs.swift.service." + name
    sc.hadoopConfiguration.set(prefix + ".auth.url", "https://identity.open.softlayer.com" + "/v3/auth/tokens")
    sc.hadoopConfiguration.set(prefix + ".auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(prefix + ".tenant", "5ad9a5281587486bb731c8ad05ddcb6b")
    sc.hadoopConfiguration.set(prefix + ".username", "e038894d793a4c81a9e319a7d25df6c8")
    sc.hadoopConfiguration.set(prefix + ".password", "EZyT{i0j#a6q-,sS")
    sc.hadoopConfiguration.setInt(prefix + ".http.port", 8080)
    sc.hadoopConfiguration.set(prefix + ".region", "dallas")
    sc.hadoopConfiguration.setBoolean(prefix + ".public", false)
}

// you can choose any name
val name = "keystone"
setHadoopConfig9433696f0f574c02aa81541a1538139f(name)



In [16]:
// READING DATA FROM CSV FILE STORED IN HDFS
val HDFSloadOptions = Map(
    Connectors.HdfsBigInsights.URL -> "https://bi-hadoop-prod-4030.bi.services.us-south.bluemix.net:8443/gateway/default/webhdfs/v1/",
    Connectors.HdfsBigInsights.USERNAME -> "varunj12",
    Connectors.HdfsBigInsights.PASSWORD -> "floridatampa1",
    Connectors.HdfsBigInsights.SOURCE_FILE_NAME -> "/user/varunj12/DATASET_BIG_DATA.csv",
    Connectors.HdfsBigInsights.SOURCE_FILE_FORMAT -> "csv",
    Connectors.HdfsBigInsights.SOURCE_INFER_SCHEMA -> "1",
     "source.invalidDataHandling" -> "column")

val Data_File = spark.read.format("com.ibm.spark.discover").options(HDFSloadOptions).load()

In [17]:
// Creating DataFrame
// val Data_File = spark.read.format("com.ibm.spark.discover").options(HDFSloadOptions).load()
Data_File.printSchema()
Data_File.show()   



root
 |-- airline_name: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- label: integer (nullable = true)

+-------------+--------------------+--------+-----+
| airline_name|               Tweet|   cabin|label|
+-------------+--------------------+--------+-----+
|adria-airways|Outbound flight F...| Economy|    1|
|adria-airways|Two short hops ZR...|Business|    1|
|adria-airways|Flew Zurich-Ljubl...| Economy|    1|
|adria-airways|Adria serves this...|Business|    1|
|adria-airways|WAW-SKJ Economy. ...| Economy|    0|
|adria-airways|Sarajevo-Frankfur...| Economy|    1|
|adria-airways|I had flights fro...| Economy|    1|
|adria-airways|LJU to FRA and ba...| Economy|    1|
|adria-airways|On my Ljubljana -...|Business|    1|
|adria-airways|Flights from LJU ...| Economy|    1|
|adria-airways|I was very satisf...| Economy|    1|
|adria-airways|I was on JP650 th...| Economy|    1|
|adria-airways|VIE-LJU LJU-MUC A...| Economy|    1|
|adria

In [18]:
Data_File.createOrReplaceTempView("airlinedata")

var results = spark.sql("SELECT * FROM airlinedata ")
results.show(5)

+-------------+--------------------+--------+-----+
| airline_name|               Tweet|   cabin|label|
+-------------+--------------------+--------+-----+
|adria-airways|Outbound flight F...| Economy|    1|
|adria-airways|Two short hops ZR...|Business|    1|
|adria-airways|Flew Zurich-Ljubl...| Economy|    1|
|adria-airways|Adria serves this...|Business|    1|
|adria-airways|WAW-SKJ Economy. ...| Economy|    0|
+-------------+--------------------+--------+-----+
only showing top 5 rows



In [19]:
var labeledTweet = results.withColumn("label",results("label").cast("double"))
// labeledTweet = results.withColumn("Rec",results.Rec.cast("double"))
labeledTweet.printSchema()
labeledTweet.sample(false,0.003,10L).show()
val Array(train_set, test_set) = labeledTweet.randomSplit(Array(0.7, 0.3), seed = 1234L)
train_set.printSchema()
train_set.show()
test_set.show()
train_set.cache()
test_set.cache()
labeledTweet.cache()

val train_count = train_set.count()
val test_count = test_set.count()

println (train_count)
println (test_count)

root
 |-- airline_name: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- label: double (nullable = true)

+--------------------+--------------------+-----------+-----+
|        airline_name|               Tweet|      cabin|label|
+--------------------+--------------------+-----------+-----+
|     aegean-airlines|As always excelle...|    Economy|  1.0|
|          aeromexico|Flew Aeromexico f...|    Economy|  0.0|
|          aer-lingus|Dublin - Santiago...|    Economy|  1.0|
|          aer-lingus|We had a terrible...|    Economy|  0.0|
|aeroflot-russian-...|London-Moscow-Ban...|    Economy|  0.0|
|           air-china|I flew RT from LA...|    Economy|  0.0|
|           air-china|London to Beijing...|    Economy|  0.0|
|           air-india|SYD-DEL-BOM via M...|    Economy|  1.0|
|          air-france|Flew MAN-CDG-HAV ...|   Business|  1.0|
|       air-mauritius|LHR-MRU April 13:...|    Economy|  1.0|
|           air-malta|Malta-Gatwi

In [20]:
val tokenizer=new Tokenizer().setInputCol("Tweet").setOutputCol("words")

val wordsDataFrame = tokenizer.transform(labeledTweet)

In [21]:
val remover = new StopWordsRemover().setInputCol("words").setOutputCol("filtered")

val wordsDataFrame1=remover.transform(wordsDataFrame)
wordsDataFrame1.show(truncate=false)

+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
val hashingTF = new HashingTF().setInputCol("filtered").setOutputCol("rawFeatures").setNumFeatures(1000)
val featurizedData = hashingTF.transform(wordsDataFrame1)
featurizedData.show()

+-------------+--------------------+--------+-----+--------------------+--------------------+--------------------+
| airline_name|               Tweet|   cabin|label|               words|            filtered|         rawFeatures|
+-------------+--------------------+--------+-----+--------------------+--------------------+--------------------+
|adria-airways|Outbound flight F...| Economy|  1.0|[outbound, flight...|[outbound, flight...|(1000,[10,18,47,6...|
|adria-airways|Two short hops ZR...|Business|  1.0|[two, short, hops...|[two, short, hops...|(1000,[149,161,16...|
|adria-airways|Flew Zurich-Ljubl...| Economy|  1.0|[flew, zurich-lju...|[flew, zurich-lju...|(1000,[69,110,161...|
|adria-airways|Adria serves this...|Business|  1.0|[adria, serves, t...|[adria, serves, 1...|(1000,[1,25,47,94...|
|adria-airways|WAW-SKJ Economy. ...| Economy|  0.0|[waw-skj, economy...|[waw-skj, economy...|(1000,[7,38,73,11...|
|adria-airways|Sarajevo-Frankfur...| Economy|  1.0|[sarajevo-frankfu...|[sarajev

In [23]:
val idf = new IDF().setInputCol("rawFeatures").setOutputCol("features")
val idfModel = idf.fit(featurizedData)

In [24]:
val lr = new LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)


In [25]:
val pipeline = new Pipeline().setStages(Array(tokenizer,remover,hashingTF,idf,lr))

In [26]:
println (tokenizer.explainParams())
println ("-----------")
println (remover.explainParams())
println ("-----------")
println (hashingTF.explainParams())
println ("-----------")
println (idf.explainParams())
println ("-----------")
println (lr.explainParams())
println ("-----------")
println (pipeline.explainParams())

inputCol: input column name (current: Tweet)
outputCol: output column name (default: tok_68c4b86f415d__output, current: words)
-----------
caseSensitive: whether to do a case-sensitive comparison over the stop words (default: false)
inputCol: input column name (current: words)
outputCol: output column name (default: stopWords_2da73328a20f__output, current: filtered)
stopWords: the words to be filtered out (default: [Ljava.lang.String;@cd9ef7ba)
-----------
binary: If true, all non zero counts are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts (default: false)
inputCol: input column name (current: filtered)
numFeatures: number of features (> 0) (default: 262144, current: 1000)
outputCol: output column name (default: hashingTF_236a1d0c182d__output, current: rawFeatures)
-----------
inputCol: input column name (current: rawFeatures)
minDocFreq: minimum number of documents in which a term should appear for filtering (default: 

In [27]:
println ("Logistic regression features column=" + lr.getFeaturesCol)
println ("logistic regression label column=")
println (lr.getLabelCol)
println ("Logistic regression threshold=")
println (lr.getThreshold)

Logistic regression features column=features
logistic regression label column=
label
Logistic regression threshold=
0.5


In [29]:
val model = pipeline.fit(train_set)
// Training the model on the training set

val predictions = model.transform(test_set)
// applying the model to the test set
predictions.printSchema()
predictions.select("airline_name","Tweet","cabin","filtered","probability","prediction","label").sample(false,0.01,10L).show(10)


root
 |-- airline_name: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- label: double (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)

+--------------------+--------------------+--------+--------------------+--------------------+----------+-----+
|        airline_name|               Tweet|   cabin|            filtered|         probability|prediction|label|
+--------------------+--------------------+--------+--------------------+--------------------+----------+-----+
|     aegean-airlines|JIK-ATH-LHR on 11...| Economy|[jik-ath-lhr, 11,...|[0.46618491114030...|       1.0|  1.0|
|     a

In [4]:
predictions.select("airline_name","Tweet","cabin","filtered","probability","prediction","label").sample(false,0.01,10L).show(10)
// Displaying Predictions against label column
val evaluator = new BinaryClassificationEvaluator().setMetricName("areaUnderROC")
// Evaluation of the results through the area under the Receiver operating characterstics curve
println ("Area under ROC curve:For Training Set-" + evaluator.evaluate(model.transform(train_set)))
println ("Area under ROC curve:For Predictions-" + evaluator.evaluate(predictions))

+--------------------+--------------------+--------+--------------------+--------------------+----------+-----+
|        airline_name|               Tweet|   cabin|            filtered|         probability|prediction|label|
+--------------------+--------------------+--------+--------------------+--------------------+----------+-----+
|     aegean-airlines|JIK-ATH-LHR on 11...| Economy|[jik-ath-lhr, 11,...|[0.46618491114030...|       1.0|  1.0|
|     aegean-airlines|Traveled on Aegea...| Economy|[traveled, aegean...|[0.46618491114030...|       1.0|  1.0|
|          aer-lingus|Flew with them ye...|Business|[flew, yesterday,...|[0.46618491114030...|       1.0|  0.0|
|aeroflot-russian-...|Bangkok - Moscow ...|Business|[bangkok, -, mosc...|[0.46618491114030...|       1.0|  1.0|
|aerolineas-argent...|Buenos Aires to B...| Economy|[buenos, aires, b...|[0.46618491114030...|       1.0|  1.0|
|aerolineas-argent...|SYD-EZE no online...| Economy|[syd-eze, online,...|[0.46618491114030...|       1.0

In [18]:
val paramGrid = new ParamGridBuilder().addGrid(hashingTF.numFeatures,Array(1000,10000,100000)).addGrid(idf.minDocFreq,Array(0,10,100)).build()

In [19]:
val cv = new CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(2)

In [20]:
val cvModel = cv.fit(train_set)
evaluator.evaluate(cvModel.transform(train_set))

0.5

In [21]:
// As the area under the ROC curve is very low we will try to use Naive Bayes

// Defining Naive bayes model
val nb = new NaiveBayes()
// Adding naive bayes to the pipeline
val pipeline_nb= new Pipeline().setStages(Array(tokenizer,remover,hashingTF,idf,nb))

In [22]:
val model_nb=pipeline_nb.fit(train_set)

In [23]:
val predictions_nb=model_nb.transform(test_set)

In [3]:
predictions_nb.select("airline_name","Tweet","cabin","filtered","probability","prediction","label").sample(false,0.01,10L).show(12)


+--------------------+--------------------+--------+--------------------+--------------------+----------+-----+
|        airline_name|               Tweet|   cabin|            filtered|         probability|prediction|label|
+--------------------+--------------------+--------+--------------------+--------------------+----------+-----+
|     aegean-airlines|JIK-ATH-LHR on 11...| Economy|[jik-ath-lhr, 11,...|[2.58468191680351...|       1.0|  1.0|
|     aegean-airlines|Traveled on Aegea...| Economy|[traveled, aegean...|[0.99324803696214...|       0.0|  1.0|
|          aer-lingus|Flew with them ye...|Business|[flew, yesterday,...|[0.99998808551675...|       0.0|  0.0|
|aeroflot-russian-...|Bangkok - Moscow ...|Business|[bangkok, -, mosc...|[1.69096964368576...|       1.0|  1.0|
|aerolineas-argent...|Buenos Aires to B...| Economy|[buenos, aires, b...|[0.94589368626632...|       0.0|  1.0|
|aerolineas-argent...|SYD-EZE no online...| Economy|[syd-eze, online,...|[0.99999999999997...|       0.0

In [25]:
val evaluator_nb = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")


// cross validation for naive bayes
val cv_nb = new CrossValidator().setEstimator(pipeline_nb).setEvaluator(evaluator_nb).setEstimatorParamMaps(paramGrid).setNumFolds(2)

In [26]:
// Evaluating Naive bayes
evaluator_nb.evaluate(model_nb.transform(train_set))

0.8074882995319813

In [27]:
evaluator_nb.evaluate(model_nb.transform(test_set))

0.8020417929494337

In [28]:
// Applying cross validation to the training set
val cvModel_nb = cv_nb.fit(train_set)
evaluator_nb.evaluate(cvModel_nb.transform(train_set))


0.8573756283584677

In [29]:
val predictions_nb_after_cv = cvModel_nb.transform(test_set)
evaluator_nb.evaluate(predictions_nb_after_cv)

0.8307545063008455

In [30]:
predictions_nb_after_cv.createOrReplaceTempView("Final_Prediction_Temp")

val Final_Data=spark.sql("select airline_name,Tweet,cabin,prediction from Final_Prediction_Temp")
Final_Data.createOrReplaceTempView("Final_Prediction")
Final_Data.show(3)
spark.sql("select * from Final_Prediction").show()

+-------------+--------------------+--------+----------+
| airline_name|               Tweet|   cabin|prediction|
+-------------+--------------------+--------+----------+
|adria-airways|Adria serves this...|Business|       1.0|
|adria-airways|BEG-LJU-BEG with ...| Economy|       0.0|
|adria-airways|BRU-LJU-BRU. Outw...| Economy|       1.0|
+-------------+--------------------+--------+----------+
only showing top 3 rows

+---------------+--------------------+--------+----------+
|   airline_name|               Tweet|   cabin|prediction|
+---------------+--------------------+--------+----------+
|  adria-airways|Adria serves this...|Business|       1.0|
|  adria-airways|BEG-LJU-BEG with ...| Economy|       0.0|
|  adria-airways|BRU-LJU-BRU. Outw...| Economy|       1.0|
|  adria-airways|I fly Adria quite...| Economy|       1.0|
|  adria-airways|LGW-LJU in Y clas...| Economy|       1.0|
|  adria-airways|LGW-LJU return. O...| Economy|       1.0|
|  adria-airways|LJU-FRA and FRA-L...| Econom

In [59]:
Final_Data.write.option("header", "true").csv("swift://project1." + name + "/final_file.csv")

1


Name: org.apache.spark.SparkException
Message: Job aborted.
StackTrace:   at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand$$anonfun$run$1.apply$mcV$sp(InsertIntoHadoopFsRelationCommand.scala:149)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand$$anonfun$run$1.apply(InsertIntoHadoopFsRelationCommand.scala:115)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand$$anonfun$run$1.apply(InsertIntoHadoopFsRelationCommand.scala:115)
  at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:57)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:115)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
  at org.apache.spark.sql.execution.command

In [44]:
val warehouseLocation = "spark-warehouse"

val spark = SparkSession.builder().appName("Spark Hive Example").config("spark.sql.warehouse.dir", warehouseLocation).enableHiveSupport().getOrCreate()

import spark.implicits._
import spark.sql
val sqlCtx= new HiveContext(sc)


In [30]:
spark.sql("select * from Final_Prediction where airline_name like 'air-india%' and prediction = 1.0 ").show()
spark.sql("select * from Final_Prediction where airline_name like 'adria-airways%' and cabin like 'Economy%' and prediction=0.0 ").show()


+------------+--------------------+--------+----------+
|airline_name|               Tweet|   cabin|prediction|
+------------+--------------------+--------+----------+
|   air-india|AI 102 from JFK t...| Economy|       1.0|
|   air-india|AI 112 Heathrow t...| Economy|       1.0|
|   air-india|AI 301 3 Feb 2014...| Economy|       1.0|
|   air-india|AI 347 30 Sep 201...| Economy|       1.0|
|   air-india|AI 55 Kochi to BO...| Economy|       1.0|
|   air-india|AI 639 BOM-BLR an...| Economy|       1.0|
|   air-india|AI 667/668 betwee...| Economy|       1.0|
|   air-india|AI143 from CDG to...| Economy|       1.0|
|   air-india|AI310 BOM-DEL. Pa...| Economy|       1.0|
|   air-india|Ageing A-320 from...| Economy|       1.0|
|   air-india|Air India (AI130)...| Economy|       1.0|
|   air-india|B787 Dreamliner (...| Economy|       1.0|
|   air-india|BKK-DEL segment. ...|Business|       1.0|
|   air-india|BKK-DEL-BHX on 9t...| Economy|       1.0|
|   air-india|BOM-DXB-BOM Airbu...| Economy|    

In [46]:
println ("Total number of airlines")
Data_File.select("airline_name").distinct.count()


Total number of airlines


371

In [47]:
spark.sql("select count(*) from Final_Prediction GROUP BY airline_name").take(4).foreach(println)


[60]
[43]
[137]
[2]


In [48]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


In [49]:
// Total number of reviews per airline
val Total_Reviews_per_airline=spark.sql("select airline_name as Airline ,count(*) as Total_reviews from Final_Prediction group by airline_name ")
Total_Reviews_per_airline.show(5)



+--------------------+-------------+
|             Airline|Total_reviews|
+--------------------+-------------+
|aeroflot-russian-...|           60|
|          air-europa|           43|
|virgin-atlantic-a...|          137|
|    atlantic-airways|            2|
|      qantas-airways|          190|
+--------------------+-------------+
only showing top 5 rows



In [50]:
%%brunel data('Total_Reviews_per_airline') bubble size(Total_reviews) color(Airline) label(Airline) legends(none) tooltip(Airline,Total_reviews)

In [51]:
// Ratio of positive to Negative Review

val all_negative = spark.sql("select airline_name, count(*) as No_of_negative_reviews from Final_Prediction where prediction=0.0 group by airline_name")
all_negative.createOrReplaceTempView("all_negative_table")
val all_positive = spark.sql("select airline_name, count(*) as No_of_positive_reviews from Final_Prediction where prediction=1.0 group by airline_name")
all_positive.createOrReplaceTempView("all_positive_table")

val all_reviews_count = spark.sql("select an.airline_name,an.No_of_negative_reviews,ap.No_of_positive_reviews from all_negative_table an inner join all_positive_table ap on an.airline_name = ap.airline_name")
all_reviews_count.createOrReplaceTempView("all_reviews_count_table")

val review_ratio = spark.sql("select airline_name,(No_of_positive_reviews/No_of_negative_reviews) as Ratio from all_reviews_count_table")
review_ratio.printSchema()

root
 |-- airline_name: string (nullable = true)
 |-- Ratio: double (nullable = true)



In [41]:
%%brunel data('review_ratio')  bar x(airline_name) y(ratio) sort(ratio) effect(enter)

In [55]:
var temtotal= spark.sql("select cabin, Count(*) as tcoun from Final_Prediction group by cabin")
temtotal.createOrReplaceTempView("temtotal_table")
var tempositive= spark.sql("select cabin, Count(*) as pcount from Final_Prediction where prediction=1 group by cabin")
tempositive.createOrReplaceTempView("tempositive_table")
var tempjoin=  spark.sql("select tt.cabin,tt.tcoun,tp.pcount from temtotal_table tt join tempositive_table tp on  tt.cabin = tp.cabin")
tempjoin.createOrReplaceTempView("tempjoin_table")
var Percentage_of_positive_reviews_per_cabin = spark.sql("select cabin, sum(pcount/tcoun) as count from tempjoin_table group by cabin")
Percentage_of_positive_reviews_per_cabin.show()

+---------------+------------------+
|          cabin|             count|
+---------------+------------------+
|    First Class|0.6926406926406926|
|Premium Economy|0.6134453781512605|
|       Business|0.7882719252724442|
|        Economy|0.5122172859450727|
+---------------+------------------+



In [58]:
// This displays which cabin class has most positive reviews and most negative reviews

%%brunel data('Percentage_of_positive_reviews_per_cabin') stack polar y(count) color(cabin) label(cabin) tooltip(count)